# Let's now build our first neural network!
- Before that, install Keras first.
- 데이터도 다운받으시고용

### 들어가기에 앞서 알아야할 딥러닝 관련 Libraries.
- Theano, Tensorflow : 수학적으로 신경망 관련 계산수행을 하게 해주는 libraries로써 CPU,GPU같은 컴퓨터 자원들을 사용자의 지정으로 사용하게 해줄수 있지만 Keras에 비교해 신경망 설계를 from scratch 해야한다.
- Keras : 위 두 라이브러리들의 기능들을 포함하고 있으며 신경망 관련 build할때 매우 간편하게 도와주는 라이브러리.

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values

In [3]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Preprocess done. Lets build Neural Network!

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


- Sequential : 그래프가 아닌, 딥러닝 모델을 Sequence of Layers로 Initialize 해준다.
- Dense : 레이어를 추가할때 Dense를 사용하여 추가시켜주기 위함이다.

In [0]:
classifier = Sequential() #NN Initialization

- 아무것도 존재하지 않는 모델을 생성했으니 이제 input, hidden, output레이어들을 넣어줘야 합니다.
- 인풋 노드의 수는 몇개여야 할까요?
- 히든 노드의 수는?
- Dense 메소드를 통해 히든레이어를 생성할수있는데, 이때 input_dim 을 통해 input노드의 개수에 맞는 input 레이어 까지 같이 생성하게됩니다.

In [8]:
#첫번째 히든레이어 생성(with input layer)
#여기선 히든노드의 개수를 input+output의 평균값으로 조정하였음.applies initial weights with uniform distribution
classifier.add(Dense(units=6,kernel_initializer = 'uniform',activation = 'relu',input_dim=11))


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#두번째 히든레이어 생성
#이건 두번째 히든레이어 생성하는거기 때문에 이미 첫번째 히든레이어에서 몇개의 인풋을 받을지 우리 모델이 알고있으므로 input_dim을 specify할필요가 없음
classifier.add(Dense(units=6,kernel_initializer = 'uniform',activation = 'relu')) 

In [0]:
#아웃풋 레이어 생성
#binary classification문제이기 때문에 predicted y가 확률적 값으로 나와야 하므로 Logistic Regression에서 쓰이는 sigmoid activation function을 사용
classifier.add(Dense(units=1,kernel_initializer = 'uniform',activation = 'sigmoid')) 

In [0]:
#모델 컴파일
#SGD의 일종인 아담 옵티마이저 사용, loss function은 output layer의 활성함수가 sigmoid이기 때문에 logistic regression에서 사용되는,
#log-loss function을 사용.(a.k.a categorical cross entropy)근데 label이 두개밖에안되기때문에 binary cross entropy.
classifier.compile(optimizer = 'adam',loss='binary_crossentropy', metrics=['accuracy'])

- 설계가 끝났으니 드디어 우리 데이터를 신경망에 집어넣어봅시다.
- epoch : 데이터셋 전체의 단위.즉 전체 데이터셋을 총 몇번돌릴거냐를 뜻함.
- batch : 가중치 업데이트를 데이터 몇개를 돌릴때마다 해줄지에 대한 단위를 뜻함.

In [12]:
classifier.fit(X_train, y_train, epochs=100, batch_size=10)
#r u ready?

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 6s 743us/step - loss: 0.4838 - acc: 0.7954
Epoch 2/100
8000/8000 [==============================] - 3s 425us/step - loss: 0.4269 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 3s 421us/step - loss: 0.4213 - acc: 0.8020
Epoch 4/100
8000/8000 [==============================] - 3s 419us/step - loss: 0.4177 - acc: 0.8239
Epoch 5/100
8000/8000 [==============================] - 3s 421us/step - loss: 0.4158 - acc: 0.8281
Epoch 6/100
8000/8000 [==============================] - 3s 423us/step - loss: 0.4142 - acc: 0.8292
Epoch 7/100
8000/8000 [==============================] - 3s 422us/step - loss: 0.4128 - acc: 0.8306
Epoch 8/100
8000/8000 [==============================] - 3s 420us/step - loss: 0.4118 - acc: 0.8324
Epoch 9/100
8000/8000 [==============================] - 3s 422us/step - loss: 0.4104 - acc: 0.8345
Epoch 10/100
8000/8000 [============================

지금까지 train 데이터셋을 돌린걸까요 test를 돌린걸까용?

In [0]:
y_pred = classifier.predict(X_test)

In [14]:
y_pred

array([[0.18668869],
       [0.3498744 ],
       [0.16894227],
       ...,
       [0.18823317],
       [0.13762504],
       [0.10647824]], dtype=float32)

지금 이 y_pred는 뭘 뜻하는걸까요? (Logistic Regression 을 다시 떠올려봅시다)

In [15]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1546,   49],
       [ 269,  136]])

In [17]:
(1546+136)/2000

0.841

Congratulations! 이제 우린 딥러닝 할줄 안다고 해도 됩니다!